In [26]:
import cv2

In [27]:
from IPython.display import display,Image

In [28]:
import ipywidgets.widgets as widgets

In [29]:
import face_recognition
import os
import pickle

In [30]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [31]:
scale = 0.333

In [32]:
# Start the video capture
cam=cv2.VideoCapture(1)

[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [33]:
# Open the training data
with open('/home/ubuntu/docs/train.pkl', 'rb') as tdf:
    names = pickle.load(tdf)
    encodings = pickle.load(tdf)

In [34]:
# Start the IPython display
display_handle=display(None, display_id=True)    

None

In [35]:
try:
    while True:
        # Read the next frame
        _, frame = cam.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image

        # Convert to RGB
        frameSmall = cv2.resize(frame,(0,0), fx=scale, fy=scale)
        frameRGB = cv2.cvtColor(frameSmall, cv2.COLOR_BGR2RGB)

        # Find faces
        facePositions = face_recognition.face_locations(frameRGB)
        allEncodings = face_recognition.face_encodings(frameRGB, facePositions)
        for (top,right,bottom,left),face_encoding in zip(facePositions,allEncodings):
            name = 'Unknown'
            matches = face_recognition.compare_faces(encodings, face_encoding)
            if True in matches:
                matchIdx = matches.index(True)
                name = names[matchIdx]
                print('Recognized', name)
            top = int(top / scale)
            right = int(right / scale)
            bottom = int(bottom / scale)
            left = int(left / scale)
            cv2.rectangle(frame, (left,top), (right,bottom), (255,0,255), 2)
            cv2.putText(frame, name, (left,top - 6), font, 0.75, (255,0,255), 2)
            
        # Display as jpeg image
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    cam.release()
    display_handle.update(None)        